### 이상치
- 결측치, 범위에서 벗어나는 데이터, 값이 크게 벗어나는 데이터
- 분석 모델의 성능을 떨어뜨리거나 결과에 악영향을 미치는 경우가 발생
- 이상치를 발견하면 확인 후 특정한 값을 대체, 삭제
- IQR 방식
    - matplotlib 에서 boxplot의 이상치 결정 방법
    - Q3(3사분위수)와 Q1(1사분위수)를 이용해서 범위 지정
    - IQR = Q3 - Q1
    - 이상치의 경계 (Q3 + 1.5 * IQR) (Q1 - 1.5 * IQR)
    - 많은 데이터를 이상치 판단할 수 있다.
    - 1.5 값을 조정해서 이상치의 범위를 변경

In [1]:
# 사이킷 런 설치
# pip install scikit-learn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine

In [3]:
# 데이터셋 로드
wine_load = load_wine()

In [4]:
wine_load

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]], shape=(178, 13)),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
# wine_load에 data를 이용하여 데이터프레임을 생성
wine = pd.DataFrame(wine_load['data'], columns=wine_load['feature_names'])
wine.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [7]:
wine['class'] = wine_load['target']

In [9]:
wine.columns

Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline', 'class'],
      dtype='object')

In [10]:
wine['color_intensity'].describe()

count    178.000000
mean       5.058090
std        2.318286
min        1.280000
25%        3.220000
50%        4.690000
75%        6.200000
max       13.000000
Name: color_intensity, dtype: float64

In [18]:
q_1, q_3 = np.percentile(wine['color_intensity'], [25, 75])

In [20]:
print(q_1)
print(q_3)

3.2199999999999998
6.2


In [21]:
iqr = q_3 - q_1
upper_whis = q_3 + (1.5 * iqr)
lower_whis = q_1 - (1.5 * iqr)
print(upper_whis, lower_whis)

10.670000000000002 -1.2500000000000009


- iqr을 이용한 이상치 판단 함수 생성
    - 매개변수 2개
        - DataFrame형태의 데이터
        - 기준이 되는 컬럼(갯수가 가변)
        - cnt (범위 기본값을 1.5)
    - 기준이 되는 컬럼의 개수만큼 반복 실행
        - 컬럼의 데이터를 기준으로 1사분위수, 3사분위수를 변수에 저장
        - iqr 변수를 생성
        - 상단의 경계, 하단의 경계를 지정
        - 데이터프레임에서 해당하는 경계를 이용하여 이상치를 출력
        - 딕셔너리에 대입 (key값은 컬럼의 이름, value는 데이터프레임)
    - 딕셔너리 데이터를 되돌려준다.

In [25]:
def outliers_iqr(_df, *_cols, _cnt = 1.5):
    # 빈 딕셔너리 데이터를 생성
    result = dict()
    # _cols는 데이터의 타입? -> tuple
    for col in _cols:
        # 1사분위, 3사분위 데이터를 생성
        q_1, q_3 = np.percentile( _df[col], [25, 75] )
        # iqr 생성
        iqr = q_3 - q_1
        # 상단 경계 하단 경계 생성
        upper_whis = q_3 + ( _cnt * iqr )
        lower_whis = q_1 - ( _cnt * iqr )
        print(f"{col}컬럼의 상단의 경계는 {upper_whis}이고 하단의 경계는 {lower_whis}이다.")
        # 상단 경계와 하단 경계를 이용하여 이상치 데이터를 추출
        outlier = _df[ (_df[col] > upper_whis) | (_df[col] < lower_whis) ]
        # 빈 딕셔너리에 대입
        result[col] = outlier
    return result

In [26]:
wine.columns

Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline', 'class'],
      dtype='object')

In [27]:
outliers = outliers_iqr(wine, 'color_intensity', 'proanthocyanins')

color_intensity컬럼의 상단의 경계는 10.670000000000002이고 하단의 경계는 -1.2500000000000009이다.
proanthocyanins컬럼의 상단의 경계는 3.0이고 하단의 경계는 0.20000000000000018이다.


In [28]:
outliers['color_intensity']

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
151,12.79,2.67,2.48,22.0,112.0,1.48,1.36,0.24,1.26,10.80,0.48,1.47,480.0,2
158,14.34,1.68,2.70,25.0,98.0,2.80,1.31,0.53,2.70,13.00,0.57,1.96,660.0,2
159,13.48,1.67,2.64,22.5,89.0,2.60,1.10,0.52,2.29,11.75,0.57,1.78,620.0,2
166,13.45,3.70,2.60,23.0,111.0,1.70,0.92,0.43,1.46,10.68,0.85,1.56,695.0,2


In [29]:
outliers['proanthocyanins']

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
95,12.47,1.52,2.20,19.0,162.0,2.50,2.27,0.32,3.28,2.6,1.16,2.63,937.0,1
110,11.46,3.74,1.82,19.5,107.0,3.18,2.58,0.24,3.58,2.9,0.75,2.81,562.0,1
